# `logistic_regression_without_race_pairwise_winners.ipynb`

### Author: Anthony Hein

#### Last updated: 11/20/2021

# Overview:

Use the datasets
* `X_train_preprocess_without_race.csv`
* `X_dev_preprocess_without_race.csv`
* `X_test_preprocess_without_race.csv`

and the targets in
* `X_train_pairwise_winner_labels.csv`
* `X_dev_pairwise_winner_labels.csv`
* `X_test_pairwise_winner_labels.csv`

to make a logistic regression model that tries to predict the pairwise winner between each pair of runners.

---

## Setup

In [1]:
from datetime import datetime
import git
import os
from typing import List
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
BASE_DIR = git.Repo(os.getcwd(), search_parent_directories=True).working_dir
BASE_DIR

'/Users/anthonyhein/Desktop/SML310/project'

---

## Load `X_train_preprocess_without_race.csv`

In [3]:
X_train = pd.read_csv(f"{BASE_DIR}/data/analysis/X_train_preprocess_without_race.csv", low_memory=False)
X_train.head()

,horse1_age,horse1_saddle,horse1_decimalPrice,horse1_isFav,horse1_outHandicap,horse1_RPR,horse1_weight,horse1_jockey_d_last_race,horse1_jockey_d_first_race,horse1_jockey_prev_1_position,...,horse2_jockey_prev_3_position_rain,horse2_jockey_prev_1_finishing_time_ratio_rain,horse2_jockey_prev_2_finishing_time_ratio_rain,horse2_jockey_prev_3_finishing_time_ratio_rain,horse2_jockey_prev_1_position_rhum,horse2_jockey_prev_2_position_rhum,horse2_jockey_prev_3_position_rhum,horse2_jockey_prev_1_finishing_time_ratio_rhum,horse2_jockey_prev_2_finishing_time_ratio_rhum,horse2_jockey_prev_3_finishing_time_ratio_rhum
0,0.200000,0.230769,0.196970,0.0,0.0,0.73125,0.692308,0.007631,0.236544,0.100,...,0.050,0.805104,0.780139,0.783438,0.100,0.025,0.025,0.805104,0.799642,0.780139
1,0.400000,0.038462,0.072222,0.0,0.0,0.73125,0.743590,0.005050,0.236544,0.100,...,0.100,0.801416,0.781238,0.794160,0.075,0.050,0.075,0.801416,0.800487,0.785703
2,0.133333,0.423077,0.098485,0.0,0.0,0.65000,0.705128,0.000740,0.237283,0.025,...,0.100,0.799909,0.780139,0.786816,0.025,0.100,0.025,0.799642,0.806486,0.780139
3,0.133333,0.423077,0.098485,0.0,0.0,0.65000,0.705128,0.000740,0.237283,0.025,...,0.075,0.800176,0.781623,0.781870,0.050,0.075,0.050,0.801163,0.801416,0.780963
4,0.133333,0.192308,0.090278,0.0,0.0,0.66250,0.730769,0.000740,0.234655,0.050,...,0.100,0.799642,0.783601,0.785468,0.125,0.100,0.025,0.803191,0.805104,0.780139


In [4]:
X_train.shape

(800666, 144)

---

## Load `X_dev_preprocess_without_race.csv`

In [5]:
X_dev = pd.read_csv(f"{BASE_DIR}/data/analysis/X_dev_preprocess_without_race.csv", low_memory=False)
X_dev.head()

,horse1_age,horse1_saddle,horse1_decimalPrice,horse1_isFav,horse1_outHandicap,horse1_RPR,horse1_weight,horse1_jockey_d_last_race,horse1_jockey_d_first_race,horse1_jockey_prev_1_position,...,horse2_jockey_prev_3_position_rain,horse2_jockey_prev_1_finishing_time_ratio_rain,horse2_jockey_prev_2_finishing_time_ratio_rain,horse2_jockey_prev_3_finishing_time_ratio_rain,horse2_jockey_prev_1_position_rhum,horse2_jockey_prev_2_position_rhum,horse2_jockey_prev_3_position_rhum,horse2_jockey_prev_1_finishing_time_ratio_rhum,horse2_jockey_prev_2_finishing_time_ratio_rhum,horse2_jockey_prev_3_finishing_time_ratio_rhum
0,0.285714,0.105263,0.200000,0.0,0.0,0.608392,0.769231,0.001516,0.629407,0.250,...,0.200,0.828972,0.837641,0.843255,0.075,0.050,0.275,0.834583,0.828972,0.853796
1,0.285714,0.210526,0.157143,0.0,0.0,0.258741,0.769231,0.000445,0.286087,0.075,...,0.075,0.826039,0.826039,0.842799,0.025,0.025,0.075,0.826039,0.826039,0.842799
2,0.285714,0.210526,0.157143,0.0,0.0,0.258741,0.769231,0.000445,0.286087,0.075,...,0.125,0.826413,0.841758,0.844367,0.125,0.050,0.275,0.843955,0.826413,0.841758
3,0.285714,0.210526,0.157143,0.0,0.0,0.258741,0.769231,0.000445,0.286087,0.075,...,0.050,0.832401,0.832114,0.828366,0.100,0.075,0.050,0.832401,0.837064,0.828519
4,0.285714,0.210526,0.157143,0.0,0.0,0.258741,0.769231,0.000445,0.286087,0.075,...,0.175,0.857255,0.835266,0.834946,0.150,0.025,0.200,0.857255,0.826039,0.837392


In [6]:
X_dev.shape

(228766, 144)

---

## Load `X_test_preprocess_without_race.csv`

In [7]:
X_test = pd.read_csv(f"{BASE_DIR}/data/analysis/X_test_preprocess_without_race.csv", low_memory=False)
X_test.head()

,horse1_age,horse1_saddle,horse1_decimalPrice,horse1_isFav,horse1_outHandicap,horse1_RPR,horse1_weight,horse1_jockey_d_last_race,horse1_jockey_d_first_race,horse1_jockey_prev_1_position,...,horse2_jockey_prev_3_position_rain,horse2_jockey_prev_1_finishing_time_ratio_rain,horse2_jockey_prev_2_finishing_time_ratio_rain,horse2_jockey_prev_3_finishing_time_ratio_rain,horse2_jockey_prev_1_position_rhum,horse2_jockey_prev_2_position_rhum,horse2_jockey_prev_3_position_rhum,horse2_jockey_prev_1_finishing_time_ratio_rhum,horse2_jockey_prev_2_finishing_time_ratio_rhum,horse2_jockey_prev_3_finishing_time_ratio_rhum
0,0.230769,0.333333,0.03268,0.0,0.0,0.344828,0.74359,0.001053,0.49501,0.225,...,0.200,0.865544,0.847264,0.853154,0.025,0.200,0.300,0.840533,0.850856,0.864844
1,0.230769,0.333333,0.03268,0.0,0.0,0.344828,0.74359,0.001053,0.49501,0.225,...,0.150,0.864781,0.840533,0.855519,0.025,0.025,0.025,0.840533,0.840533,0.840533
2,0.230769,0.333333,0.03268,0.0,0.0,0.344828,0.74359,0.001053,0.49501,0.225,...,0.075,0.838534,0.843337,0.846422,0.200,0.100,0.125,0.853412,0.842583,0.847859
3,0.230769,0.333333,0.03268,0.0,0.0,0.344828,0.74359,0.001053,0.49501,0.225,...,0.250,0.851685,0.849673,0.854624,0.150,0.250,0.200,0.849673,0.854624,0.858051
4,0.230769,0.333333,0.03268,0.0,0.0,0.344828,0.74359,0.001053,0.49501,0.225,...,0.250,0.847892,0.842776,0.862423,0.175,0.075,0.275,0.853634,0.844529,0.862845


In [8]:
X_test.shape

(114392, 144)

---

## Load Pairwise Winner Labels

In [9]:
X_train_pairwise_winner_labels = np.loadtxt(f"{BASE_DIR}/data/analysis/x_train_pairwise_winner_labels.csv",
                                            dtype=int,
                                            delimiter=',')

In [10]:
X_dev_pairwise_winner_labels = np.loadtxt(f"{BASE_DIR}/data/analysis/X_dev_pairwise_winner_labels.csv",
                                          dtype=int,
                                          delimiter=',')

In [11]:
X_test_pairwise_winner_labels = np.loadtxt(f"{BASE_DIR}/data/analysis/X_test_pairwise_winner_labels.csv",
                                           dtype=int,
                                           delimiter=',')

---

## Logistic Regression Model

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import PredefinedSplit, GridSearchCV

## Fixing Random Seed for Reproducibility
np.random.seed(0)

In [47]:
## Hyperparameter Space
space = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'C': [0.01, 0.1, 10, 100],
}

In [34]:
X_train_plus_dev = pd.concat([X_train, X_dev])
X_train_plus_dev.shape

(1029432, 144)

In [35]:
X_train_plus_dev_pairwise_winner_labels = np.concatenate(
    (X_train_pairwise_winner_labels, X_dev_pairwise_winner_labels)
)
X_train_plus_dev_pairwise_winner_labels

array([1, 0, 1, ..., 1, 1, 1])

In [36]:
len(X_train), len(X_dev)

(800666, 228766)

In [37]:
test_fold = np.concatenate(
    (np.full(len(X_train), -1),
     np.zeros(len(X_dev)))
)
test_fold

array([-1., -1., -1., ...,  0.,  0.,  0.])

In [38]:
## Predefined Split
ps = PredefinedSplit(test_fold)
ps

PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0]))

In [48]:
lr = LogisticRegression(max_iter=1000)

In [49]:
lr = GridSearchCV(lr, space, verbose=25, refit=False, cv=ps)

In [50]:
search = lr.fit(X_train_plus_dev.to_numpy(), X_train_plus_dev_pairwise_winner_labels)

Fitting 1 folds for each of 48 candidates, totalling 48 fits
[CV 1/1; 1/48] START C=0.01, penalty=none, solver=newton-cg.....................


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 1/1; 1/48] END C=0.01, penalty=none, solver=newton-cg;, score=0.943 total time= 2.4min
[CV 1/1; 2/48] START C=0.01, penalty=none, solver=lbfgs.........................


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 1/1; 2/48] END C=0.01, penalty=none, solver=lbfgs;, score=0.943 total time= 1.3min
[CV 1/1; 3/48] START C=0.01, penalty=none, solver=liblinear.....................
[CV 1/1; 3/48] END C=0.01, penalty=none, solver=liblinear;, score=nan total time=   0.8s
[CV 1/1; 4/48] START C=0.01, penalty=l1, solver=newton-cg.......................
[CV 1/1; 4/48] END C=0.01, penalty=l1, solver=newton-cg;, score=nan total time=   0.8s
[CV 1/1; 5/48] START C=0.01, penalty=l1, solver=lbfgs...........................
[CV 1/1; 5/48] END C=0.01, penalty=l1, solver=lbfgs;, score=nan total time=   0.8s
[CV 1/1; 6/48] START C=0.01, penalty=l1, solver=liblinear.......................
[CV 1/1; 6/48] END C=0.01, penalty=l1, solver=liblinear;, score=0.942 total time=15.1min
[CV 1/1; 7/48] START C=0.01, penalty=l2, solver=newton-cg.......................
[CV 1/1; 7/48] END C=0.01, penalty=l2, solver=newton-cg;, score=0.927 total time=  47.6s
[CV 1/1; 8/48] START C=0.01, penalty=l2, solver=lbfgs..................

...

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 1/1; 37/48] END C=100, penalty=none, solver=newton-cg;, score=0.943 total time= 1.8min
[CV 1/1; 38/48] START C=100, penalty=none, solver=lbfgs.........................


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 1/1; 38/48] END C=100, penalty=none, solver=lbfgs;, score=0.943 total time= 1.0min
[CV 1/1; 39/48] START C=100, penalty=none, solver=liblinear.....................
[CV 1/1; 39/48] END C=100, penalty=none, solver=liblinear;, score=nan total time=   0.8s
[CV 1/1; 40/48] START C=100, penalty=l1, solver=newton-cg.......................
[CV 1/1; 40/48] END C=100, penalty=l1, solver=newton-cg;, score=nan total time=   0.7s
[CV 1/1; 41/48] START C=100, penalty=l1, solver=lbfgs...........................
[CV 1/1; 41/48] END C=100, penalty=l1, solver=lbfgs;, score=nan total time=   0.7s
[CV 1/1; 42/48] START C=100, penalty=l1, solver=liblinear.......................
[CV 1/1; 42/48] END C=100, penalty=l1, solver=liblinear;, score=0.943 total time=161.7min
[CV 1/1; 43/48] START C=100, penalty=l2, solver=newton-cg.......................
[CV 1/1; 43/48] END C=100, penalty=l2, solver=newton-cg;, score=0.943 total time= 1.2min
[CV 1/1; 44/48] START C=100, penalty=l2, solver=lbfgs.................

/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
24 fits failed out of a total of 48.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 464, in _check_solver
    raise ValueError("penalty='none' is not

In [51]:
print(f"Best Score: {search.best_score_}")
print(f"Best Hyperparameters: {search.best_params_}")

Best Score: 0.9431558885498719
Best Hyperparameters: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


In [53]:
pd.DataFrame(search.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,params,split0_test_score,mean_test_score,std_test_score,rank_test_score
0,142.769847,0.0,0.704565,0.0,0.01,none,newton-cg,"{'C': 0.01, 'penalty': 'none', 'solver': 'newt...",0.943042,0.943042,0.0,10
1,75.759495,0.0,0.076401,0.0,0.01,none,lbfgs,"{'C': 0.01, 'penalty': 'none', 'solver': 'lbfgs'}",0.943095,0.943095,0.0,4
2,0.767871,0.0,0.000000,0.0,0.01,none,liblinear,"{'C': 0.01, 'penalty': 'none', 'solver': 'libl...",NaN,NaN,NaN,46
3,0.802443,0.0,0.000000,0.0,0.01,l1,newton-cg,"{'C': 0.01, 'penalty': 'l1', 'solver': 'newton...",NaN,NaN,NaN,45
4,0.778186,0.0,0.000000,0.0,0.01,l1,lbfgs,"{'C': 0.01, 'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN,NaN,44


In [54]:
pd.DataFrame(search.cv_results_).to_csv(
    f"{BASE_DIR}/data/analysis/logistic_regression/without_race_with_weather.csv", index=False
)

In [13]:
import pickle

lr = LogisticRegression(
    max_iter=1000,
    C=100,
    penalty='l2',
    solver='lbfgs',
).fit(X_train.to_numpy(), X_train_pairwise_winner_labels)

with open(f"{BASE_DIR}/data/analysis/logistic_regression/without_race_with_weather.pkl",'wb') as f:
    pickle.dump(lr, f)

---